P

In [22]:
import pandas as pd
from sage.all import *

vhod = "SA_1f_vse.csv"
izhod = "subpath_2del_lastnosti.csv"
def is_cactus(G):
    cycles = G.cycle_basis()
    k = len(cycles)
    for i in range(k):
        C1 = cycles[i]
        for j in range(i+1, k):
            C2 = cycles[j]
            if len(set(C1) & set(C2)) > 1:
                return False
    return True

def all_undirected_cycles(G):
    DG = DiGraph(G)  # naredi usmerjen graf iz ne-usmerjenega
    raw_cycles = DG.all_simple_cycles()

    seen = set()
    cycles = []

    for cyc in raw_cycles:
        if len(cyc) < 3:
            continue

        # normalizacija cikla (rotacije + obrni smer)
        n = len(cyc)
        # najnižji vrh kot začetek
        start = min(range(n), key=lambda i: cyc[i])
        r1 = tuple(cyc[(start + i) % n] for i in range(n))
        r2 = tuple(reversed(r1))
        key = min(r1, r2)

        if key not in seen:
            seen.add(key)
            cycles.append(list(key))

    return cycles
def is_PTC(G):
    cycles = all_undirected_cycles(G)
    k = len(cycles)
    for i in range(k):
        C1 = cycles[i]
        for j in range(i+1, k):
            C2 = cycles[j]
            if len(set(C1) & set(C2)) > 2:
                return False
    return True

def obdelaj_csv(vhod, izhod, mode="both"):
    """
    vhod: pot do CSV
    izhod: kam shraniti novi CSV
    mode: "min", "max", ali "both"
    """

    df = pd.read_csv(vhod, dtype=str, keep_default_na=False)

    # priprava novih stolpcev
    if mode in ("min", "both"):
        df["PTC_min"] = ""
        df["KAKTUS_min"] = ""

    if mode in ("max", "both"):
        df["PTC_max"] = ""
        df["KAKTUS_max"] = ""

    for i, row in df.iterrows():

        # ------- MIN -------
        if mode in ("min", "both"):
            g6_min = row["graph6_min"]
            try:
                G_min = Graph(g6_min)
            except:
                G_min = None

            if G_min is None:
                df.at[i, "PTC_min"] = "false"
                df.at[i, "KAKTUS_min"] = "false"
            else:
                df.at[i, "PTC_min"] = str(is_PTC(G_min)).lower()
                df.at[i, "KAKTUS_min"] = str(is_cactus(G_min)).lower()

        # ------- MAX -------
        if mode in ("max", "both"):
            g6_max = row["graph6_max"]
            try:
                G_max = Graph(g6_max)
            except:
                G_max = None

            if G_max is None:
                df.at[i, "PTC_max"] = "false"
                df.at[i, "KAKTUS_max"] = "false"
            else:
                df.at[i, "PTC_max"] = str(is_PTC(G_max)).lower()
                df.at[i, "KAKTUS_max"] = str(is_cactus(G_max)).lower()

    df.to_csv(izhod, index=False)
    return df

obdelaj_csv(vhod, izhod, mode="both")



,n,µ(G),min p_n(G),max p_n(G),slika_min,slika_max,graph6_min,graph6_max,PTC_min,KAKTUS_min,PTC_max,KAKTUS_max
0,3,1,9,9,slike_min/graf_min_mu1_n3.png,slike_max/graf_max_mu1_n3.png,Bw,Bw,true,true,true,true
1,4,1,15,16,slike_min/graf_min_mu1_n4.png,slike_max/graf_max_mu1_n4.png,CV,C],true,true,true,true
2,5,1,22,25,slike_min/graf_min_mu1_n5.png,slike_max/graf_max_mu1_n5.png,DC{,DUW,true,true,true,true
3,6,1,30,36,slike_min/graf_min_mu1_n6.png,slike_max/graf_max_mu1_n6.png,E?bw,EEh_,true,true,true,true
4,7,1,39,49,slike_min/graf_min_mu1_n7.png,slike_max/graf_max_mu1_n7.png,F?AFw,FCp`_,true,true,true,true
...,...,...,...,...,...,...,...,...,...,...,...,...
205,26,8,855,43820,slike_min/graf_min_mu8_n26.png,slike_max/graf_max_mu8_n26.png,Y???P@IGW@A?OG?G?_ChH?GA??O@A?@O???G???G???_??...,Y?OOA?A`??C??@?C?P?O??OGGI?a??GO?_@GO??CC?_?C_...,true,true,false,false
206,27,8,898,47973,slike_min/graf_min_mu8_n27.png,slike_max/graf_max_mu8_n27.png,Z???P@IGW@A?OG?G?_ChH?GA??O@A?@O???G???G???_??...,Z?OOA?A???????GC?P?O??OGGI?a??GO?_?GO??CC?_?C_...,true,true,false,false
207,28,8,942,52292,slike_min/graf_min_mu8_n28.png,slike_max/graf_max_mu8_n28.png,[???P@IGW@A?OG?G?_ChH?GA??O@A?@O???G???G???_??...,[?OOB?A???????GC?P?O???GGI?a??GO?_?G?D?CC?_?C_...,true,true,false,false
208,29,8,987,56861,slike_min/graf_min_mu8_n29.png,slike_max/graf_max_mu8_n29.png,\???P@IGW@A?OG?G?_ChH?GA??O@A?@O???G???G???_??...,\?OOA?A???????GC?@?O?G?GGI?A??GO?A?G?C?CC?_?C_...,true,true,false,false


Spodnja funkcija preveri ali velja implikacija na najinih rezultatih knorov izrek
UGOTOVITEV: Izrek drzi, saj vse grafe za max_subpath_number, ki so kaktusni vrne tudi true da je PTC graf

In [23]:
import pandas as pd

def preveri_kaktus_ptc_max(ime_csv):
    df = pd.read_csv(ime_csv, dtype=str, keep_default_na=False)

    # normalizacija na lowercase
    kaktus = df["KAKTUS_max"].str.lower()
    ptc    = df["PTC_max"].str.lower()

    # vrstice, kjer je kaktus_max = true, ptc_max pa NI true
    mask = (kaktus == "true") & (ptc != "true")

    if mask.any():
        print("Najdene napake (KAKTUS_max = true, PTC_max != true):")
        print(df[mask])
    else:
        print("Vse OK: kjer je KAKTUS_max = true, je tudi PTC_max = true.")

    return df[mask]

# primer klica
napake = preveri_kaktus_ptc_max("subpath_2del_lastnosti.csv")


Vse OK: kjer je KAKTUS_max = true, je tudi PTC_max = true.
